In [1]:
# DRY Don't repeat yourself
# Six words to tell you what the function does

def double(x):
    return 2 * x

print(double)

<function double at 0x7fdd99f5b6d0>


In [ ]:
twotimes = double # Treat function as an object
print(type(twotimes))
print(twotimes)

<class 'function'>
<function double at 0x7fdd99f5b6d0>


In [ ]:
print(double(17))
print(twotimes(12))
print(twotimes(4) == double(4)) 

34
24
True


In [ ]:
list(map(twotimes, [2,3,4])) 
# Not twotimes() as referring to function as an object, not trying to execute the function
# Called 'function composition'

[4, 6, 8]

In [9]:
def some_function():
    print("Ran some function")

def wrapper(func_to_run):
    print("Ran wrapper...")
    func_to_run()
    print("Finished wrapper.")

wrapper(some_function)

Ran wrapper...
Ran some function
Finished wrapper.


In [ ]:
def my_decorator(func_to_run):
    def wrapper():
        print("Wrapper Started")
        func_to_run()
        print("Wrapper Ended")
    
    return wrapper

f = my_decorator(some_function)
f() # F is now shorthand for this specifically wrapped function

Wrapper Started
Ran some function
Wrapper Ended


In [15]:
# Python provides syntatical sugar to do this

@my_decorator
def hello():
    print("Hello")

hello()

f()

Wrapper Started
Hello
Wrapper Ended
Wrapper Started
Ran some function
Wrapper Ended


In [16]:
### GLOBAL VARIABLES ###

In [ ]:
def do_task():
    x = 10

x = 5
do_task()
print(x)
# Python looks at local, enclosing, etc. namespace and then prints the first x it finds.abs

5


In [21]:
def do_task():
    global x
    x = 10

x = 5
do_task()
print(x)
# Imagine do_task is passed to parsl
# If x global, don't know execution order, so some do_tasks use global x and some use local x. 
# Cannot determine a priori result of a concurrent function that uses global variables

10


In [ ]:
a = 3
def do_stuff(b):
    return b * a

do_stuff(6)
# Nesting functions creates implicit linkage
# This is not always bad!

18

In [ ]:
# Functional programming languages treat functions as first class objects

# Pure functions rely only on stuff in its paramater list and use immutable data types
# No matter when called, if no globals in program
# Function that has side effect (e.g. saves something to disc) is not a pure function
    # Can run into deadlocks and race conditions

In [26]:
global x

def f(x):
    return x * x

f(x)
x = 7
f(x)


49

In [43]:
from concurrent.futures import ProcessPoolExecutor
import time

def hello(i):
    print(i, "Hello")
    print(i, "World")


executor = ProcessPoolExecutor()
futures = [executor.submit(hello, i) for i in range(3)]

for future in futures:
    future.result()

# Screen is a shared resource!

120  HelloHello
 2
 Hello0
World1
  WorldWorld



In [ ]:
# Use a lock to fix this
from concurrent.futures import ProcessPoolExecutor
import time
import multiprocessing

def hello(i, lock):
    with lock: # Nothing else can happen between time of getting lock, until after with block
        print(i, "Hello")
        print(i, "World")

lock = multiprocessing.Manager().Lock()
executor = ProcessPoolExecutor()
futures = [executor.submit(hello, i, lock) for i in range(3)]

for future in futures:
    future.result() # Still don't know order, as only locks that hello and world work together
    

0 Hello
0 World
2 Hello
2 World
1 Hello
1 World
